<a href="https://colab.research.google.com/github/Zoe3399/Study_DeepLearning/blob/main/Ex10_mlp_cnn__%EA%B0%9C%2C%EA%B3%A0%EC%96%91%EC%9D%B4_%EC%9D%B4%EC%A7%84%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 드라이브 연동
# 경로 딥러닝 폴더 이동

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/.commit/

/content/drive/MyDrive/.딥러닝


In [ ]:
# 개, 고양이 데이터 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = np.load('./Data/np_cat_vs_dog.npz')
data

In [ ]:
len(data)

4

In [ ]:
# 각각의 변수에 데이터 분리해서 담아주기
X_train = data['X_train']
X_test = data['X_test']

y_train = data['y_train']
y_test = data['y_test']

# 크기 확인
print("훈련셋 : ", X_train.shape, y_train.shape)
print("테스트셋 :", X_test.shape, y_test.shape)

훈련셋 :  (2000, 224, 224, 3) (2000,)
테스트셋 : (1000, 224, 224, 3) (1000,)


# mlp 모델 생성

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten # 1차원 펴주는 도구
from tensorflow.keras.callbacks import EarlyStopping # 조기 학습 중단 도구

In [ ]:
# 1. 모델 설계
# 뼈대 구축
mlp_model = Sequential()
# 입력(3-> 1)
mlp_model.add(Flatten(input_shape = (224,224,3))) #행, 열, 채널수(컬러3)

# 중간층
mlp_model.add(Dense(256,activation = 'relu'))
mlp_model.add(Dense(128,activation = 'relu'))
mlp_model.add(Dense(64,activation = 'relu'))

# 출력층(분류?)
mlp_model.add(Dense(1, activation = 'sigmoid'))
mlp_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 150528)            0         
                                                                 
 dense (Dense)               (None, 256)               38535424  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 38576641 (147.16 MB)
Trainable params: 38576641 (147.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 번역 compile
# pc가 이해하도록 번역하는 작업
# loss, optimizer, metrics 설정
# 이진 분류
mlp_model.compile(loss = 'binary_crossentropy',
                  optimizer = 'adam',
                  metrics = ['accuracy'])

In [ ]:
from IPython.core.magic import validate_type
from traitlets.traitlets import validate
#조기학습 중단 설정
# 검증 정확도 기준, 5번 기다림
f_ea = EarlyStopping(monitor='val_accuracy', patience = 5)

In [ ]:
# 학습, 검증데이터 분리 30%, batch_size 64, 반복횟수 50, 조기학습중단 연결
mlp_his = mlp_model.fit(X_train, y_train,
                        validation_split = 0.3,
                        batch_size = 64,
                        epochs = 50,
                        callbacks = [f_ea])

Epoch 1/50
22/22 [==============================] - 7s 41ms/step - loss: 2849.1799 - accuracy: 0.5771 - val_loss: 3674.2876 - val_accuracy: 0.0000e+00
Epoch 2/50
22/22 [==============================] - 1s 24ms/step - loss: 2011.3843 - accuracy: 0.6157 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/50
22/22 [==============================] - 0s 23ms/step - loss: 1690.7632 - accuracy: 0.5629 - val_loss: 1455.5076 - val_accuracy: 0.0100
Epoch 4/50
22/22 [==============================] - 0s 22ms/step - loss: 271.6706 - accuracy: 0.6507 - val_loss: 0.7515 - val_accuracy: 0.9917
Epoch 5/50
22/22 [==============================] - 1s 23ms/step - loss: 576.2346 - accuracy: 0.5643 - val_loss: 0.8565 - val_accuracy: 0.9900
Epoch 6/50
22/22 [==============================] - 1s 27ms/step - loss: 126.7212 - accuracy: 0.6621 - val_loss: 248.0348 - val_accuracy: 0.2017
Epoch 7/50
22/22 [==============================] - 1s 25ms/step - loss: 135.0614 - accuracy: 0.6500 - val_loss: 6.8920 - v

In [ ]:
# test 데이터 활용해서 평가하기
# 모델.score(문제, 답) - > 머신 러닝 모델 평가하는 함수 (sklearn)
# 모델.evaluate(문제, 답) - > 딥러닝 모델 평가하는 함수 (tensorflow.keras)
mlp_model.evaluate(X_test, y_test)

32/32 [==============================] - 0s 6ms/step - loss: 138.2711 - accuracy: 0.5280


[138.2710723876953, 0.527999997138977]

#### cnn 모델 생성
  1. 모델 설계 (cnn층)
  2. 모델 학습 및 평가 방법 설정
  3. 모델 학습 및 시각화
  4. 모델 평가 및 예측

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [ ]:
#cnn 신경망 구조를 설계
# 뼈대 생성
cnn_model = Sequential()

# 대상에 집중 ! -> 중요한 특성에 집중 ! -> 특성 추출
# 특성 추출부(층) - conv(합성곱)
cnn_model.add(Conv2D(32, (3,3), padding = 'same', input_shape = (224, 224, 3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size = 2))

cnn_model.add(Conv2D(64, (3,3), padding = 'same', activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size = 2))

cnn_model.add(Conv2D(128, (3,3), padding = 'same', activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size = 2))

# 개, 고양이 분류부(층) - mlp(입력(1차원펴주는 기능), 중간, 출력층(이진분류))
cnn_model.add(Flatten()) # 1차원으로 펴!
cnn_model.add(Dense(512, activation = 'relu')) # 중간층
cnn_model.add(Dense(1, activation = 'sigmoid')) # 출력층

# 모델 정보 확인
cnn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 112, 112, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 56, 56, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 28, 28, 128)      

In [ ]:
# complie
cnn_model.compile(loss = 'binary_crossentropy',
                  optimizer = 'adam',
                  metrics = ['accuracy'])

# fit
cnn_his = cnn_model.fit(X_train, y_train,
                        validation_split = 0.3,
                        epochs = 50, batch_size = 64,
                        callbacks = [f_ea])

Epoch 1/50
22/22 [==============================] - 13s 199ms/step - loss: 129.2666 - accuracy: 0.6086 - val_loss: 1.4386 - val_accuracy: 0.0033
Epoch 2/50
22/22 [==============================] - 2s 104ms/step - loss: 0.5775 - accuracy: 0.7336 - val_loss: 1.1904 - val_accuracy: 0.1883
Epoch 3/50
22/22 [==============================] - 2s 102ms/step - loss: 0.4041 - accuracy: 0.8236 - val_loss: 1.1390 - val_accuracy: 0.4333
Epoch 4/50
22/22 [==============================] - 2s 98ms/step - loss: 0.2824 - accuracy: 0.8800 - val_loss: 2.7546 - val_accuracy: 0.0750
Epoch 5/50
22/22 [==============================] - 2s 97ms/step - loss: 0.2017 - accuracy: 0.9243 - val_loss: 2.3208 - val_accuracy: 0.2700
Epoch 6/50
22/22 [==============================] - 2s 98ms/step - loss: 0.1224 - accuracy: 0.9529 - val_loss: 3.6134 - val_accuracy: 0.2833
Epoch 7/50
22/22 [==============================] - 2s 107ms/step - loss: 0.0955 - accuracy: 0.9729 - val_loss: 3.6418 - val_accuracy: 0.1850
Epoch 

In [ ]:
# 과대적합 보여짐
# 왜 이렇게 학습했을까?
# cnn 특성에 집중 -> val 예측 성능은 떨어지는 상태
# 데이터는 복잡한 데이터일 수 있음
# 반대로 모델은 너무 단순할 수 있음
# 평가
cnn_model.evaluate(X_test, y_test)[1] # 60%

32/32 [==============================] - 1s 19ms/step - loss: 2.1362 - accuracy: 0.6010


0.6010000109672546

## 2023.09.13 수업 시작